In [2]:
%pip install torch torchvision torchaudio transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 4.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 6.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [transformers] [transformers]ub]
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import pandas as pd
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

# 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

# CSV 파일 로드
df = pd.read_csv('/Volumes/1TB/스인재/esgc team project/data/NewsResult_20250423-20250723 뉴스.csv')

print(f"데이터 크기: {df.shape}")
print(f"컬럼명: {list(df.columns)}")
print("\n첫 번째 기사 제목:")
print(df['제목'].iloc[0])
print("\n첫 번째 기사 본문 일부:")
print(df['본문'].iloc[0][:200] + "...")

text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITI

데이터 크기: (10952, 19)
컬럼명: ['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3', '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드', '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부']

첫 번째 기사 제목:
이 대통령, 6개 시군 특별재난 지역 선포 “재난에 음주가무 엄히 단속”

첫 번째 기사 본문 일부:
[앵커]

 이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포했습니다.

 이 대통령은 재난 상황에서 부적절한 공직자들의 처신 문제도 거론했는데, 엄혹한 현장에서 음주가무를 즐기는 정신 나간 경우도 있었다며 엄히 단속하라고 주문했습니다.

 보도에 이희연 기자입니다.

 [리포트]

경기도 가평과 충남 서산 예산.....


'TV가 없는 집도 많아지고 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다.'

In [6]:
def summarize_text(text):
    """
    주어진 텍스트를 요약하는 함수
    """
    if pd.isna(text) or len(text.strip()) == 0:
        return "텍스트가 없습니다."
    
    # 텍스트가 너무 길면 일부만 사용 (토큰 제한 때문)
    if len(text) > 1000:
        text = text[:1000]
    
    try:
        # 토큰화
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
        
        # 요약 생성
        summary_ids = model.generate(torch.tensor([input_ids]), max_length=150, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        
        return summary
    except Exception as e:
        return f"요약 생성 중 오류 발생: {str(e)}"

# 첫 번째 기사 요약 테스트
first_article = df['본문'].iloc[0]
print("원본 기사:")
print(first_article[:500] + "...")
print("\n요약:")
summary = summarize_text(first_article)
print(summary)

원본 기사:
[앵커]

 이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포했습니다.

 이 대통령은 재난 상황에서 부적절한 공직자들의 처신 문제도 거론했는데, 엄혹한 현장에서 음주가무를 즐기는 정신 나간 경우도 있었다며 엄히 단속하라고 주문했습니다.

 보도에 이희연 기자입니다.

 [리포트]

경기도 가평과 충남 서산 예산.....

요약:
이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포했는데, 엄혹한 현장에서 음주가무를 즐기는 공직자들의 처신 문제도 거론했는데, 엄혹한 현장에서 음주가무를 즐기는 정신 나간 경우도 있었다며 엄히 단속하라고 주문하였다.
이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포했는데, 엄혹한 현장에서 음주가무를 즐기는 공직자들의 처신 문제도 거론했는데, 엄혹한 현장에서 음주가무를 즐기는 정신 나간 경우도 있었다며 엄히 단속하라고 주문하였다.


In [7]:
# 샘플 데이터로 여러 기사 요약 (처음 5개 기사)
sample_size = 5
sample_df = df.head(sample_size).copy()

print(f"{sample_size}개 기사 요약 중...")
summaries = []

for idx, row in sample_df.iterrows():
    print(f"기사 {idx + 1} 요약 중...")
    title = row['제목']
    content = row['본문']
    summary = summarize_text(content)
    
    summaries.append({
        '원본_제목': title,
        '언론사': row['언론사'],
        '일자': row['일자'],
        '요약': summary
    })

# 결과를 DataFrame으로 변환
summary_df = pd.DataFrame(summaries)

# 결과 출력
for i, row in summary_df.iterrows():
    print(f"\n{'='*50}")
    print(f"기사 {i+1}")
    print(f"제목: {row['원본_제목']}")
    print(f"언론사: {row['언론사']} | 일자: {row['일자']}")
    print(f"요약: {row['요약']}")
    print(f"{'='*50}")

# 요약 결과를 CSV로 저장
summary_df.to_csv('/Volumes/1TB/스인재/esgc team project/news_summaries.csv', index=False, encoding='utf-8-sig')
print(f"\n요약 결과가 'news_summaries.csv' 파일로 저장되었습니다.")

5개 기사 요약 중...
기사 1 요약 중...
기사 2 요약 중...
기사 2 요약 중...
기사 3 요약 중...
기사 3 요약 중...
기사 4 요약 중...
기사 4 요약 중...
기사 5 요약 중...
기사 5 요약 중...

기사 1
제목: 이 대통령, 6개 시군 특별재난 지역 선포 “재난에 음주가무 엄히 단속”
언론사: KBS | 일자: 20250723
요약: 이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포했는데, 엄혹한 현장에서 음주가무를 즐기는 공직자들의 처신 문제도 거론했는데, 엄혹한 현장에서 음주가무를 즐기는 정신 나간 경우도 있었다며 엄히 단속하라고 주문하였다.

기사 2
제목: 신경호 결심 공판 열려 검찰 “징역 3년 요구”
언론사: KBS | 일자: 20250723
요약: 신경호 강원도교육감에 대한 결심공판이 어제(22일) 열렸고, 검찰은 신 교육감에게 징역형을 내려야한다고 주장했고, 검찰은 징역형을 내려야한다고 주장했다.

기사 3
제목: ‘계엄 옹호’ 강준욱 자진 사퇴 최동석 ‘2차 가해’ 발언 등 논란 확산
언론사: KBS | 일자: 20250723
요약: 새 정부 인사들의 과거 발언이 잇따라 논란이 되면서 최동석 인사혁신처장도 박원순 전 시장 사건을 두고 한 과거 발언이 2차 가해 논란에 섰다.

기사 4
제목: 이 대통령, 강선우 등 4명 청문보고서 재송부 요청
언론사: KBS | 일자: 20250723
요약: 이재명 대통령은 국회에 강선우 여성가족부 장관 후보자를 포함해, 국방부와 국가보훈부, 통일부 장관 후보자에 대한 인사청문보고서를 내일까지 재송부해 달라고 요청했고, 이에 대해 강유정 대통령실 대변인은 기한은 24일로 요청했다고 밝히며 "이번주 내 임명을 마무리하고 신속한 국정 안정을 꾀하기 위해 기한은 오는 24일로 요청했다"고 말했다.

기사 5
제목: 6곳 특별재난지역 선포 “공직자 부적절 행위 엄히 단속”
언론사: KBS | 일자: 20250723
요약: 정부가 경기 가평